## Backtesting 

- Trades at close
- Commission = 0.125%
- SL = TP = 5%
- Cash = 1 M
- N Shares = 50
- No taxes
- No leverages
- Margin acc = 50%
- Borrow rate = 0.25%
- Time frame = 5 m

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ta

sns.set_theme()

In [48]:
data = pd.read_csv("aapl_5m_train.csv")
data.head()

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0


In [49]:
rsi_indicator = ta.momentum.RSIIndicator(data.Close, window = 20)
data['rsi'] = rsi_indicator.rsi()

In [50]:
historic = data.dropna()
historic['buy_signal'] = historic.rsi < 20
historic['sell_signal'] = historic.rsi > 80

/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_20767/3384102837.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['buy_signal'] = historic.rsi < 20
/var/folders/39/pg899vtj5774718bjd1zhn6m0000gn/T/ipykernel_20767/3384102837.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historic['sell_signal'] = historic.rsi > 80


In [51]:
COM = 0.125 / 100
SL = 0.05
TP = 0.05
BORROW_RATE = 0.25 / 100

cash = 1_000_000
n_shares = 50

In [52]:
from dataclasses import dataclass

@dataclass
class Operation:
    time: str
    price: float
    stop_loss: float
    take_profit: float
    n_shares: int
    type: str

In [53]:
active_positions: list[Operation] = []

for i, row in historic.iterrows():
    # close positions
    for position in active_positions.copy():

        if row.Close > position.take_profit or row.Close < position.stop_loss:
            # close position

            cash += row.Close * position.n_shares * (1 - COM)
            active_positions.remove(position)

    # check signal

    if not row.buy_signal:
        continue
    
    #check if you have enough cash

    if cash < row.Close * n_shares * (1 + COM):
        continue

    #discount the cost

    cash -= row.Close * n_shares * (1 + COM)
    # save the operation as active position

    active_positions.append(
        Operation(
            time=row.Datetime,
            price=row.Close,
            stop_loss=row.Close * (1 - SL),
            take_profit=row.Close * (1 + TP),
            n_shares=n_shares,
            type='LONG'
        )
    )


In [54]:
cash

1007477.8104106248